In [1]:
import warnings
warnings.filterwarnings("ignore")

import generator as g
import glob

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

# Folder Structure of Assets

* assets
    * images
        * main
            * 0000.png
            * 0001.png
            * ...
            * 0099.png
        * hidden
            * hidden.png
        * layers
            * base
                * name1_(count).png
                * name2_(count).png
            * base2
                * name3_(count).png
                * ...
            * sites
                * layer1
                    * nameX_(prob).png
                    * nameY_(prob).png
                * layer2
                    * ...
    * metadata
        * main
            * 0000.json
            * 0001.json
            * ...
            * 0099.json
        * hidden
            * hidden.json


### 1. Prepare layers folder, using your preferred method to generate the base and layer images
note:   
1. base layers are the art layer that would be included in every image generated, where each base layer has a number of different "attributes" (styles)
2. if the NFT collection generated has a total of 128 pieces, every attribute of the base layer should have a count that when totalled, equal 128
3. this count would be included in the image file name
3. site layers are the art layer that may or may not be included in the image based on the random generator, where each site layer has a number of different "attributes" (styles)
4. for sites, instead of a count, we use the probability instead totalling to 1 and included in the image file name as well

Note: if running from github, the collection will be 8 instead of 128 as a demo

### 2. Setup base and site layer info

In [3]:
path_layers = "../assets/images/layers"
pool_size = None
plot = False

base_attributes = {}
attributes = {}
attributes_prob = {}
for folder_base in sorted(glob.glob(f"{path_layers}/**")):
    folder = folder_base.split("/")[-1]
    if folder != "sites":
        attrs = [f.split("/")[-1].split(".")[0] for f in sorted(glob.glob(f"{path_layers}/{folder}/*.png"))]
        base_attributes_tmp = {}
        for attr_ in attrs:
            attr = attr_.split("_")
            base_attributes_tmp[attr_] = int(attr[-1])
        base_attributes[folder] = base_attributes_tmp
    else:
        for folder_site in sorted(glob.glob(f"{folder_base}/**")):
            folder = folder_site.split("/")[-1]
            attrs = [f.split("/")[-1][:-4] for f in sorted(glob.glob(f"{folder_base}/{folder}/*.png"))]
            attrs_prob = [float(f.split("/")[-1].split("_")[-1][:-4]) for f in sorted(glob.glob(f"{folder_base}/{folder}/*.png"))]
            assert sum(attrs_prob) == 1.0
            attributes[folder] = attrs
            attributes_prob[folder] = attrs_prob

### 3. Generate final distribution

In [4]:
final_dict = g.RGC_without_repetition(base_attributes, attributes, attributes_prob, pool_size=pool_size, plot=plot)

128 initial generation choices
128 final pool choices
128 final pool choices w/ attributes


### 4. Generate final images

In [6]:
path_base = f"{path_layers}/Background"
path_base2 = f"{path_layers}/Expression"
path_sites = f"{path_layers}/sites"
path_output = "../assets/images/main"
g.generate_final_images_from_layers(final_dict, path_base, path_base2, path_sites, path_output)

### 5. Generate promotional GIF from subset of final images
copy some images from main to demo folder to generate GIF

In [7]:
path_gifs = "../assets/images/demo.gif"
g.generate_gif_from_images("../assets/images/demo", path_gifs)

### 6. Upload `main` & `hidden` **`images`** to nft.storage: https://nft.storage/files/  via npx hardhat run scripts/upload_assets.js [UNCOMMENT data paths first]

### 7. Generate Metadata
NOTE: copy CID from uploaded images

In [8]:
### use main's CID
g.generate_metadata("<INSERT_CID>", final_dict, base_attributes, attributes, path_image="../assets/images/main", path_metadata="../assets/metadata/main", hidden=False)

### use hidden's CID
g.generate_metadata("<INSERT_CID>", final_dict, base_attributes, attributes, path_image="../assets/images/hidden", path_metadata="../assets/metadata/hidden", hidden=True)

### 6. Upload `main` & `hidden` **`metadata`** to nft.storage: https://nft.storage/files/  via npx hardhat run scripts/upload_assets.js [UNCOMMENT data paths first]

### 9. Get **metadata** main & hidden URL and go to 1_deploy.js